# A Greedy Heuristic for routing assignment 

Using the [Minimum Cost Flow Problem](/Jupyter%20notebooks/Inventory_Rebalancing_With_Minimum_Cost_Flow.ipynb) we have defined exhanges between supply and demand nodes in such a way that the net demand at each bikestations returns to 0 at the end of a given day. These exchanges are at an approximated minimum value. We now want to assign vehicles to traverse the network of bikestations to carry out these exchanges. We took a look at doing this using a [network transformation and google OR-tools](/Jupyter%20notebooks/Route_Rebalancing_Using_TSP_and_Google_ORtools.ipynb). The approach had limitations and couldn't be extended to multiple vehicles. 

Instead we will now introduce a greedy heuristic method for approximating the minimum cost route on our bipartite network which performs all the necessary exchanges for returning the demand in the  network to its original state. The solution to this problem will be useful for the bike station companies who are looking to send vehicles out to rebalance the bike stations at a low cost. 

## Greedy Heuristic 

We want a way of traversing the network so that all of the demand is satisfied at an approximated minimum cost. Consider the case of one vehicle starting at a supply node. There are one or more 'exchange' edges to demand nodes from that supply node that it can take. These 'exchange' edges are what we calculated using the minimum cost flow problem. 

So the vehicle at the supply node can travel along the shortest available exchange edge, or nearest demand node that is served from that supply node. It can make a note that this edge has been traversed and if all the exchange edges have been traversed it marks the given supply node as satisfied. 

Now the vehicle is at a demand node. It can calculate the nearest unsatisfied supply node and return to it. It then repeats the process until all of the supply nodes are satsified. 

This is the basis for our greedy heursitic, we track the unsatisfied supply nodes and traverse continue moving to the nearest edges until the problem all the supply nodes are satisfied. 

The problem is easily extendable to multiple vehicles. We simply set a maximum distance that a vehcile can travel and stop the algorithm for the vehicle once it has reached its maximum travel distance. We then start the algorithm for a new vehicle from a random supply node with the left over unsatisfied supply nodes. 


## Implementation 

We first load in the data on stationEdges, which is the edges between all nodes and their weights. 
We also load in the keyEdges which are the edges which must be traversed in order to bring the net demand in the network back to 0. See the [Minimum Cost Flow Problem](/Jupyter%20notebooks/Inventory_Rebalancing_With_Minimum_Cost_Flow.ipynb) notebook for how this was calculated. 

In [256]:
import pandas as pd 
import pickle 
import os 
import geopandas as gpd
import pyproj
import copy

stationEdges = pd.read_pickle(os.path.join(os.getcwd(),r'pickles\stationEdges.pkl'))

with open(os.path.join(os.getcwd(),r'pickles\keyEdges.pkl'), 'rb') as fp:
    keyEdges = pickle.load(fp)


We also load in the demand and supply nodes daataframes 

In [257]:
demand =  pd.read_pickle(os.path.join(os.getcwd(),r'pickles\demand.pkl'))
supply =  pd.read_pickle(os.path.join(os.getcwd(),r'pickles\supply.pkl'))

We convert the demand and supply nodes to GeoDataFrames with the london local coordinate system for easy calculation. 
We also set the index to the station id. 

In [258]:
demand = gpd.GeoDataFrame(demand,crs="EPSG:4326",geometry=demand['geometry'] )
supply= gpd.GeoDataFrame(supply,crs="EPSG:4326",geometry=supply['geometry'] )
demand = demand.set_index('id')
supply = supply.set_index('id')
demand = demand.to_crs(crs = pyproj.CRS("EPSG:27700"))
supply = supply.to_crs(crs = pyproj.CRS("EPSG:27700"))

Next we create a dictionary to track the unsatisfied nodes. 
This dictionary is in the form (supplyid: [$d_1,d_2,d_3$]), where $d_1,\dots,d_n$ are the demand nodes this suppy node must exchange with. 

We do this in the following way 

* Update unsat with the suppy nodes key and an empty array
* For each of the supply nodes append the demand nodes it is connected to along with the distance to the demand node (node,distance)
* Sort each of the unsat supply -> demand arrays by distance 
* Remove the weight to the edge 

In [259]:
sat = []
unsat = {}
for x in keyEdges.keys():
    unsat.update({x[0]:[]})
for x in keyEdges.keys():
    unsat.get(x[0]).append((x[1],keyEdges.get(x)))
for x in unsat.keys():
    unsat.get(x).sort(key = lambda x: x[1])
for x in unsat.keys():
    unsat.update({x:list(map(lambda x : x[0],unsat.get(x)))})


Lets take a look at a sample entry. We have the supply node along with the demand nodes it needs to serve sorted by distance.

In [260]:
list(unsat.keys())[0], unsat.get(list(unsat.keys())[3])

('84', ['364', '98'])

Next lets define a function that runs the heuristic for a single vehicle.

In [261]:
def greedy_route_heuristic(unsat,sat,start,max_route_length):
    cur = start 
    total_length  = 0
    route = [] 
    while len(unsat) != 0 or total_length >= max_route_length:
        available_edges =  unsat.get(cur) # get edges available from current supply node 
        to_demand = available_edges[0]    # get nearest demand node 
        cur_to_demand_length = stationEdges.loc[(cur,to_demand)]['weight'] # get distance from supply to nearest demand
        if total_length + cur_to_demand_length > max_route_length: # ensure max length not exceeded
            return unsat,sat,route,total_length
            
        ## Add supply-> demand to route 
        route.append(cur)
        route.append(to_demand)
        total_length += cur_to_demand_length

        ## Remove traversed edge from unsat array. 
        available_edges.pop(0)
        ##If array is empty mark supply node as satisfied 
        if len(available_edges) == 0:
           unsat.pop(cur)
           sat.append(cur)
        
        if len(unsat) ==0: #We just satsifed the last demand node, there is no need to return to a supply node
            continue
        else:  #Find the nearest supply node to return to add it to the route 
            demand_geom = demand.loc[to_demand].geometry
            x = supply[supply.index.isin(list(unsat.keys()))]
            distances_to_supply = x.distance(demand_geom)
            next_distance = distances_to_supply.loc[distances_to_supply.idxmin()]

            if total_length + next_distance > max_route_length: # Enforce max route for vehicle constraint 
                return unsat,sat,route,total_length
            
            next = distances_to_supply.idxmin()
            total_length += next_distance
            cur = next
        
    return unsat,sat,route,total_length
        



The function takes 
* Unsatisfied supply nodes dictionary
* Satisfied supply node array
* Supply node to start the algorithm from 
* The maximum route length 

The algorithm continues until there are no more unsatisfied nodes or the maximum route length is acheived. 

It starts with the current supply node and gets the list of demand edges that are not satisfied. 
It selects the nearest demand node from the unsatisfied edges along with the distance to that edge.
It then performs a check to see if travelling to that node will violate the maximum distance for a vehicle. 
If it does not it will add the distance to the route length and and the nodes to the route array. 

Next it removes the demand node  from the list of unsatisfied available edges for that suppyl node and if the supply node has no more available nodes it marks it as satisfied. 

Next it performs a check to see if we just satisfied the last possible supply edge, if it does it skips returning to the nearest unsatisfied supply edge as the algorithm will end on the next loop. 

Otherwise it finds the nearest unsatisfied supply node. If returning to it does not violate our maximum distance constraint it returns to it and adds the length and node ids to the route. 

Lastly it sets the next node to the nearest supply node we just calculated. 

If at anytime the algorithm terminates it returns the unsatsfied nodes, satisfied nodes along with the route and route length. 



### Solving with a single vehicle 

We run the algorithm with an excessively high maximum travel distance. This is equivelant to solving with a single vehicle. Lets see the output. 

In [262]:
unsatAfter,satAfter,route,route_length= greedy_route_heuristic(copy.deepcopy(unsat),copy.deepcopy(sat),list(unsat.keys())[0], 1300000)
unsatAfter,route_length

({}, 1273684.785954546)

We can see that all nodes are satisfied and there is a single route length of  1273684.785954546.

In [263]:
def solve_routing_greedy_with_vehicle_distance_limit(u,vehicle_max_travel):
    route_lengths = []
    routes = [] 
    curUnsat = u
    curSat = [] 
    
    while len(curUnsat) != 0:
        u,s,r,rl = greedy_route_heuristic(curUnsat,sat,list(curUnsat.keys())[0],vehicle_max_travel)
        route_lengths.append(rl)
        routes.append(r)
    return routes,route_lengths,curUnsat



In [264]:
routes,route_lengths,u =solve_routing_greedy_with_vehicle_distance_limit(copy.deepcopy(unsat), 1300000)
route_lengths


[1273684.785954546]

## Solving with multiple vehicles 

We know that a single vehicle needs to travel 1273684.785954546 meters to completly rebalance all the inventory at a particular bike station. 

The company managing the bike stations would want find a the lowest number of vehicles necessary to balance all the bike stations in a specific time frame. Since we are looking over a 24 hour period, this could be from 0000 hrs to 04000-05000 hours so between 4 to 5 hours. 

Using our greedy heuristic, we can range over different maximum vehicle lengths and calculate the number of vehicles needed to solve the problem with each length. We also can get the approximated straight line length each of these vehicles would need to travel. 

We do the following
* Starting with a base max_vehicle length of 50000, solve the routing problem with the greedy heuristic 
* Increment the max vehicle length by 50000 and solve the problem again. 
* Repeat until the maximum vehicle length is greater than 1300000 which is around what is needed to solve the problem with a single vehicle. 

In [265]:

one_vehicle_max = 1300000

max_travel_per_vehicle = [] 
number_of_vehicles_needed = [] 

for x in range(50000,one_vehicle_max+50000,50000):
    routes,route_lengths,u =solve_routing_greedy_with_vehicle_distance_limit(copy.deepcopy(unsat),x)
    max_travel_per_vehicle.append(x)
    number_of_vehicles_needed.append(len(route_lengths))
    
    

Lets see the results. 

In [266]:
print("max-vehicle-length","no-vehicles-needed")
for n,x in enumerate(range(50000,one_vehicle_max+50000,50000)):
    print(x,number_of_vehicles_needed[n])


max-vehicle-length no-vehicles-needed
50000 26
100000 13
150000 9
200000 7
250000 6
300000 5
350000 4
400000 4
450000 3
500000 3
550000 3
600000 3
650000 2
700000 2
750000 2
800000 2
850000 2
900000 2
950000 2
1000000 2
1050000 2
1100000 2
1150000 2
1200000 2
1250000 2
1300000 1


Now we have to consider the time each of these vehicles would be taking. We see that the speed limit on a restricted road is approximately [48km/h or 48000m/h](https://www.autoexpress.co.uk/car-news/98575/uk-speed-limits-everything-you-need-to-know#:~:text=UK%20speed%20limits%20for%20buses,weighing%20up%20to%207.5%20tonnes%3A&text=Restricted%20road%3A%2030mph,-%E2%80%A2&text=Single%20carriageway%3A%2050mph,-%E2%80%A2&text=Dual%20carriageway%3A%2060mph,-%E2%80%A2). The maximum time is dependant on the maximum distance travelled by each vehicle. We can calculate with a number of vehicles how long it would take. 



In [267]:
print("max-vehicle-length","no-vehicles-needed","Time-needed")
for n,x in enumerate(range(50000,one_vehicle_max+50000,50000)):
    print(x,number_of_vehicles_needed[n], x/48000 )

max-vehicle-length no-vehicles-needed Time-needed
50000 26 1.0416666666666667
100000 13 2.0833333333333335
150000 9 3.125
200000 7 4.166666666666667
250000 6 5.208333333333333
300000 5 6.25
350000 4 7.291666666666667
400000 4 8.333333333333334
450000 3 9.375
500000 3 10.416666666666666
550000 3 11.458333333333334
600000 3 12.5
650000 2 13.541666666666666
700000 2 14.583333333333334
750000 2 15.625
800000 2 16.666666666666668
850000 2 17.708333333333332
900000 2 18.75
950000 2 19.791666666666668
1000000 2 20.833333333333332
1050000 2 21.875
1100000 2 22.916666666666668
1150000 2 23.958333333333332
1200000 2 25.0
1250000 2 26.041666666666668
1300000 1 27.083333333333332


The only reasonable values that fall within our 4-5 hour time frame are between 7 to 26 vehicles. 

The following values are the most reasonable for rebalancing the bike station inventory. 

* 7 vehicles : 4.17 hrs 
* 9 vehicles : 3.125 hrs 
* 13 vehicles :2.08 hrs 

Since we are using straight line distances it is probably better to err on the side of caution and use 9-13 vehicles. It may also be more feasible to perform rebalancing every 3 -4 days rather than at the end of every single day.